# Transformer

In [1]:
!pip install keras-nightly keras-hub-nightly --upgrade -q

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras

filename = keras.utils.get_file(
    origin=(
        "https://storage.googleapis.com/download.tensorflow.org/"
        "data/shakespeare.txt"
    ),
)
shakespeare = open(filename, "r").read()

In [3]:
shakespeare[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [4]:
import tensorflow as tf

sequence_length = 100

def split_input(input, sequence_length):
    for i in range(0, len(input), sequence_length):
        yield input[i : i + sequence_length]

features = list(split_input(shakespeare[:-1], sequence_length))
labels = list(split_input(shakespeare[1:], sequence_length))
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [5]:
x, y = next(dataset.as_numpy_iterator())

In [6]:
x[:50], y[:50]

(b'First Citizen:\nBefore we proceed any further, hear',
 b'irst Citizen:\nBefore we proceed any further, hear ')

In [7]:
from keras import layers

tokenizer = layers.TextVectorization(
    standardize=None,
    split=lambda x: tf.strings.unicode_split(x, 'UTF-8'),
    output_sequence_length=sequence_length,
)

tokenizer.adapt(dataset.map(lambda text, labels: text))

In [8]:
vocabulary_size = tokenizer.vocabulary_size()
vocabulary_size

67

In [9]:
dataset = dataset.map(
    lambda features, labels: (tokenizer(features), tokenizer(labels)),
    num_parallel_calls=8,
)
training_data = dataset.shuffle(10_000).batch(64).cache()

In [10]:
embedding_dim = 256
hidden_dim = 1024

inputs = layers.Input(shape=(sequence_length,), dtype=tf.int32, name="token_ids")
x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x = layers.GRU(hidden_dim, return_sequences=True)(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
model = keras.Model(inputs, outputs, name="shakespeare")

In [11]:
model.summary(line_length=80)

Model: "shakespeare"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)            │ (None, 100)              │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ embedding (Embedding)             │ (None, 100, 256)         │        17,152 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ gru (GRU)                         │ (None, 100, 1024)        │     3,938,304 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout (Dropout)                 │ (None, 100, 1024)        │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense (Dense)                     │ (None, 100, 67)          │        68,675 │
└───────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 4,024,131 (15.35 MB)

 Trainable params: 4,024,131 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.fit(training_data, epochs=20)

Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 14s 50ms/step - loss: 2.7170 - sparse_categorical_accuracy: 0.2797
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 1.9997 - sparse_categorical_accuracy: 0.4152
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.7381 - sparse_categorical_accuracy: 0.4843
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.5862 - sparse_categorical_accuracy: 0.5249
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.4917 - sparse_categorical_accuracy: 0.5492
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.4258 - sparse_categorical_accuracy: 0.5651
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 1.3740 - sparse_categorical_accuracy: 0.5785
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.3294 - sparse_categorical_accuracy: 0.5901
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.2886 - sparse_categorical_accuracy: 0.6011
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 1

In [13]:
inputs = keras.Input(shape=(1,), dtype="int", name="token_ids")
input_state = keras.Input(shape=(hidden_dim,), name="state")

x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x, output_state = layers.GRU(hidden_dim, return_state=True)(
    x, initial_state=input_state
)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
generation_model = keras.Model(
    inputs=(inputs, input_state),
    outputs=(outputs, output_state),
)
generation_model.set_weights(model.get_weights())

In [14]:
tokens = tokenizer.get_vocabulary()
token_ids = range(vocabulary_size)
char_to_id = dict(zip(tokens, token_ids))
id_to_char = dict(zip(token_ids, tokens))

prompt = """
KING RICHARD III:
"""

In [15]:
input_ids = [char_to_id[c] for c in prompt]
state = keras.ops.zeros(shape=(1, hidden_dim))
for token_id in input_ids:
    inputs = keras.ops.expand_dims([token_id], axis=0)
    predictions, state = generation_model.predict((inputs, state), verbose=0)

In [16]:
import numpy as np

generated_ids = []
max_length = 250
for i in range(max_length):
    next_char = int(np.argmax(predictions, axis=-1)[0])
    generated_ids.append(next_char)
    inputs = keras.ops.expand_dims([next_char], axis=0)
    predictions, state = generation_model.predict((inputs, state), verbose=0)

In [17]:
output = "".join([id_to_char[token_id] for token_id in generated_ids])
print(prompt + output)


KING RICHARD III:
Richard! where is the man that slew my brother?

BIANCA:
Why, they say the letters that the sea for tears the sea
Which he did stand upon the earth.

GLOUCESTER:
Why, then the son of self-saint be a man
Than the selfsame note of your livery is a man

